In [1]:
%cd ..

c:\Users\HuyLQ15_CTV\Desktop\model_monitoring\notebook


In [2]:
import pandas as pd
import numpy as np
import requests
import zipfile
import io

from datetime import datetime, time
from sklearn import datasets, ensemble

from evidently import ColumnMapping
from evidently.report import Report
from evidently.metric_preset import DataDriftPreset, TargetDriftPreset, RegressionPreset

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
content = requests.get("https://archive.ics.uci.edu/static/public/275/bike+sharing+dataset.zip", verify=False).content
with zipfile.ZipFile(io.BytesIO(content)) as arc:
    raw_data = pd.read_csv(arc.open("hour.csv"), header=0, sep=',', parse_dates=['dteday'], index_col='dteday')

In [5]:
from datetime import time, datetime

raw_data.index = raw_data.apply(
    lambda row: datetime.combine(row.name, time(hour=int(row['hr']))), axis = 1)

# Regression Quality

* Step 1: 
* Step 2: 
* Step 3:

## Prepare data

In [7]:
target = 'cnt'
prediction = 'prediction'
numerical_features = ['temp', 'atemp', 'hum', 'windspeed', 'hr', 'weekday']
categorical_features = ['season', 'holiday', 'workingday']
column_mapping = ColumnMapping()
column_mapping.numerical_features = numerical_features
column_mapping.categorical_features = categorical_features

In [8]:
reference = raw_data.loc['2011-01-01 00:00:00':'2011-01-31 00:00:00']
current = raw_data.loc['2011-02-01 00:00:00':'2011-07-28 23:00:00']

In [9]:
current.keys()

Index(['instant', 'season', 'yr', 'mnth', 'hr', 'holiday', 'weekday',
       'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed',
       'casual', 'registered', 'cnt'],
      dtype='object')

In [10]:
current['time_stamp'] = current.index
reference['time_stamp'] = reference.index

In [11]:
reference

,instant,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt,time_stamp
2011-01-01 00:00:00,1,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0000,3,13,16,2011-01-01 00:00:00
2011-01-01 01:00:00,2,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0000,8,32,40,2011-01-01 01:00:00
2011-01-01 02:00:00,3,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0000,5,27,32,2011-01-01 02:00:00
2011-01-01 03:00:00,4,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0000,3,10,13,2011-01-01 03:00:00
2011-01-01 04:00:00,5,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0000,0,1,1,2011-01-01 04:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2011-01-30 20:00:00,661,1,0,1,20,0,0,0,2,0.26,0.2727,0.65,0.1045,3,30,33,2011-01-30 20:00:00
2011-01-30 21:00:00,662,1,0,1,21,0,0,0,2,0.24,0.2424,0.70,0.1642,3,25,28,2011-01-30 21:00:00
2011-01-30 22:00:00,663,1,0,1,22,0,0,0,2,0.24,0.2273,0.70,0.1940,2,19,21,2011-01-30 22:00:00
2011-01-30 23:00:00,664,1,0,1,23,0,0,0,2,0.24,0.2121,0.65,0.2836,5,16,21,2011-01-30 23:00:00


# NannyML

In [12]:
import nannyml as nml
import pandas as pd
from IPython.display import display

In [13]:
reference

,instant,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt,time_stamp
2011-01-01 00:00:00,1,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0000,3,13,16,2011-01-01 00:00:00
2011-01-01 01:00:00,2,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0000,8,32,40,2011-01-01 01:00:00
2011-01-01 02:00:00,3,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0000,5,27,32,2011-01-01 02:00:00
2011-01-01 03:00:00,4,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0000,3,10,13,2011-01-01 03:00:00
2011-01-01 04:00:00,5,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0000,0,1,1,2011-01-01 04:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2011-01-30 20:00:00,661,1,0,1,20,0,0,0,2,0.26,0.2727,0.65,0.1045,3,30,33,2011-01-30 20:00:00
2011-01-30 21:00:00,662,1,0,1,21,0,0,0,2,0.24,0.2424,0.70,0.1642,3,25,28,2011-01-30 21:00:00
2011-01-30 22:00:00,663,1,0,1,22,0,0,0,2,0.24,0.2273,0.70,0.1940,2,19,21,2011-01-30 22:00:00
2011-01-30 23:00:00,664,1,0,1,23,0,0,0,2,0.24,0.2121,0.65,0.2836,5,16,21,2011-01-30 23:00:00


## Continuos

In [14]:
from nannyml.distribution import ContinuousDistributionCalculator

continuous_dist = ContinuousDistributionCalculator(
    column_names=['temp'],
    timestamp_column_name = 'time_stamp',
    chunk_period='M',
)
fig = continuous_dist.fit(reference).calculate(current).filter(period='all', column_names=['temp']).plot(kind='distribution')


## Categorical

In [15]:
categorical_features

['season', 'holiday', 'workingday']

In [16]:
from nannyml.distribution import CategoricalDistributionCalculator

categorical_dist = CategoricalDistributionCalculator(
    column_names=['workingday'],
    timestamp_column_name = 'time_stamp',
    chunk_period='M',
)
categorical_dist.fit(reference).calculate(current).filter(period='all', column_names=['workingday']).plot(kind='distribution')